In [ ]:
!pip install ultralytics tqdm pillow --quiet

In [ ]:
import os
import shutil
from pathlib import Path
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import PIL.Image as Image

In [ ]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
    zip_path = f"/content/{fn}"

In [ ]:
!mkdir -p /content/dataset
!unzip -q "$zip_path" -d /content/dataset


In [ ]:
project_dir = Path("/content/yolo_project")
dataset_dir = Path("/content/dataset")
output_dir = project_dir / "dataset"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
images_dir = dataset_dir / "images"
labels_dir = dataset_dir / "labels"

all_images = sorted(list(images_dir.glob("*.jpg")) + list(images_dir.glob("*.png")))
train_imgs, temp_imgs = train_test_split(all_images, test_size=0.3, random_state=42)
val_imgs, test_imgs = train_test_split(temp_imgs, test_size=0.5, random_state=42)

splits = {"train": train_imgs, "val": val_imgs, "test": test_imgs}

def copy_split(split_name, files):
    img_dest = output_dir / split_name / "images"
    lbl_dest = output_dir / split_name / "labels"
    os.makedirs(img_dest, exist_ok=True)
    os.makedirs(lbl_dest, exist_ok=True)
    for img_path in tqdm(files, desc=f"Копируем {split_name}"):
        shutil.copy(img_path, img_dest / img_path.name)
        label_path = labels_dir / f"{img_path.stem}.txt"
        if label_path.exists():
            shutil.copy(label_path, lbl_dest / label_path.name)

for split_name, files in splits.items():
    copy_split(split_name, files)

print("✅ Train/Val/Test готов!")

In [ ]:
classes_file = dataset_dir / "classes.txt"
with open(classes_file, "r") as f:
    class_names = [line.strip() for line in f if line.strip()]
num_classes = len(class_names)
print(f"✅ Классы: {class_names}")

In [ ]:
yaml_path = project_dir / "data.yaml"
yaml_content = f"""
train: {output_dir}/train/images
val: {output_dir}/val/images
test: {output_dir}/test/images
nc: {num_classes}
names: {class_names}
"""
with open(yaml_path, "w") as f:
    f.write(yaml_content)
print(f"✅ YAML-файл создан: {yaml_path}")

In [ ]:
model = YOLO("yolov8x.pt")

model.train(
    data="/content/yolo_project/data.yaml",
    epochs=30,
    imgsz=640,
    batch=8,
    lr0=1e-3,
    patience=10,
    optimizer="Adam",
    mosaic=0.5,
    mixup=0.2,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    flipud=0.3,
    fliplr=0.5,
    degrees=15,
    shear=5,
    name="small_dataset_yolov8x"
)

print("✅ Обучение завершено!")

In [ ]:
metrics = model.val()
print("📊 Результаты валидации:", metrics)

In [ ]:
from google.colab import files

print("📤 Загрузите фото для проверки:")
uploaded = files.upload()

for filename in uploaded.keys():
    test_image_path = f"/content/{filename}"
    print(f"📷 Проверяем изображение: {test_image_path}")


    results = model.predict(
        source=test_image_path,
        conf=0.5,
        save=True,
        show=False
    )


    result_image = results[0].plot()
    display(Image.fromarray(result_image))
    print("✅ Предсказание завершено! Результат также сохранён в runs/detect/predict")